# Tapis federated authentication prototyping

### Sample Workflow (to remember)
1. Authentication client ID is shipped with software 
2. A non-authenticated tapis instance is created (only pass the tenant base url when instantiating)
3. make the get device code call with the client ID
4. navigate to webpage and enter user code
5. generate token, wait for confirmation on website
6. website confirmation, token generation
7. pass token to tapis, now authenticated as individual user

(this is all correct?)

### other things
* have a way for users to choose to use tacc account or federated OAuth at startup
* external service to serve tenant specific client IDs so that the app can be more flexible

In [11]:
import requests
import time
import pprint as pp
import datetime
from getpass import getpass
import pytz
import neo4jupyter
from py2neo import Graph
from tapipy.tapis import Tapis

In [2]:
import tapipy
tapipy.__version__

'1.3.6'

In [18]:
t.pods.create_pod(pod_id='silliness3', pod_template='template/neo4j', testarg='dinkus', description='')


command: None
creation_ts: 2023-07-03T21:24:47.692585
data_attached: []
data_requests: []
description: 
environment_variables: 

networking: 
default: 
port: 7687
protocol: tcp
url: silliness3.pods.smartfoods.develop.tapis.io
pod_id: silliness3
pod_template: template/neo4j
resources: 
cpu_limit: 2000
cpu_request: 250
mem_limit: 3072
mem_request: 256
roles_inherited: []
roles_required: []
start_instance_ts: None
status: REQUESTED
status_container: 

status_requested: ON
time_to_stop_default: 43200
time_to_stop_instance: None
time_to_stop_ts: None
update_ts: 2023-07-03T21:24:47.692949
volume_mounts: 

In [3]:
def show(res):
    try:
        pp.pprint(res.json())
    except:
        pp.pprint(res.text)

neo4jupyter.init_notebook_mode()

<IPython.core.display.Javascript object>

In [12]:
base_url = "smartfoods.develop.tapis.io"

In [13]:
start = time.time()

# Base URL for Tapis
username = str(input("username"))

# Get Tapis object if it isn't already created.
try:
    if t.base_url == base_url and t.username == username and t.access_token:
        print("Tapis object already exists.")
        if t.access_token.expires_at < datetime.datetime.now(pytz.utc):
            print("Existing Tapis token expired, getting new token.")
            raise
    else:
        print("Creating new Tapis object.")
        raise
except:
    try:
        t = Tapis(base_url = f"https://{base_url}",
                  username = username,
                  password = getpass('password'))
        t.get_tokens()
    except Exception as e:
        print(f"\nBROKEN! timeout: {time.time() - start}\n")
        raise

# V3 Headers
header_dat = {"X-Tapis-token": t.access_token.access_token,
              "Content-Type": "application/json"}

# Service URL
url = f"{base_url}/v3"                   # remote

print(time.time() - start)
print(f"base_url: {base_url}")
print(f"serv_url: {url}")

Creating new Tapis object.
5.791557550430298
base_url: smartfoods.develop.tapis.io
serv_url: smartfoods.develop.tapis.io/v3


In [4]:
Tapis().resource_dicts['authenticator']

{'openapi': '3.0.2',
 'info': {'title': 'Authenticator',
  'description': 'REST API and web server providing authentication for a Tapis v3 instance.',
  'version': '1',
  'termsOfService': 'https://tapis-project.org',
  'contact': {'name': 'Authenticator',
   'url': 'https://tapis-project.org',
   'email': 'cicsupport@tacc.utexas.edu'},
  'license': {'name': 'BSD 3',
   'url': 'https://github.com/tapis-project/authenticator'}},
 'servers': [{'url': 'http://localhost:5000',
   'description': 'Local Development'},
  {'url': 'http://{tenant_id_url}.develop.tapis.io',
   'description': 'Tapis Develop instance',
   'variables': {'tenant_id_url': {'default': 'dev',
     'description': 'The tenant_id associated with the request.'}}},
  {'url': '/',
   'description': 'catch-all server definition for other Tapis instances.'}],
 'paths': {'/v3/oauth2/hello': {'get': {'tags': ['Health Check'],
    'description': 'Logged connectivity test. No authorization required.',
    'operationId': 'hello',
 

In [7]:
jwt = t.get_access_jwt()

In [5]:
t = Tapis(base_url = f"https://{base_url}",
          download_latest_specs=True,
          resource_set='dev',
          spec_dir='/some/new/dir')

In [8]:
clients = t.authenticator.list_clients()
test_client = clients[0]

In [6]:
test_client

NameError: name 'test_client' is not defined

In [6]:
client_id = 'Oa9aD3zr2vJ0g'
client_code_info = t.authenticator.generate_device_code(client_id=client_id)
print(client_code_info)



client_id: Oa9aD3zr2vJ0g
device_code: n3r5sUkKgdKMR8v3mQANgOdjEDBoZGuKQVwDflI4
expires_in: Mon, 03 Jul 2023 20:46:05 GMT
user_code: hMTLAnWQ
verification_uri: http://smartfoods.develop.tapis.io/v3/oauth2/device?client_id=Oa9aD3zr2vJ0g


In [7]:
try:
    tokens = t.authenticator.create_token(grant_type="device_code", device_code=client_code_info.device_code, client_id=client_id)
except Exception as e:
    er = e

In [8]:
print(tokens)


access_token: 
access_token: eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiJ9.eyJqdGkiOiI4ZGJiYmYwNy1hZTk0LTQ2ZTgtOWNiOC1mYzQxZGMyMjVlMTYiLCJpc3MiOiJodHRwczovL3NtYXJ0Zm9vZHMuZGV2ZWxvcC50YXBpcy5pby92My90b2tlbnMiLCJzdWIiOiJta3l6dWJyQHRhY2NAc21hcnRmb29kcyIsInRhcGlzL3RlbmFudF9pZCI6InNtYXJ0Zm9vZHMiLCJ0YXBpcy90b2tlbl90eXBlIjoiYWNjZXNzIiwidGFwaXMvZGVsZWdhdGlvbiI6ZmFsc2UsInRhcGlzL2RlbGVnYXRpb25fc3ViIjpudWxsLCJ0YXBpcy91c2VybmFtZSI6Im1reXp1YnJAdGFjYyIsInRhcGlzL2FjY291bnRfdHlwZSI6InVzZXIiLCJleHAiOjE2OTEwMDg1ODUsInRhcGlzL2NsaWVudF9pZCI6Ik9hOWFEM3pyMnZKMGciLCJ0YXBpcy9ncmFudF90eXBlIjoiZGV2aWNlX2NvZGUiLCJ0YXBpcy9pZHBfaWQiOiJ0YWNjIiwidGFwaXMvcmVmcmVzaF9jb3VudCI6MH0.I36FeqXBCNkV5tBBKgGSP9pV5luiaIxreFiFrb7TR9csscUDTpsaxqAG1fzj-SV58djCqu-GFurf2rj7j7mYQM1BpmoTEI7VRDCl45U_PPcALq1sOcVsR4oF0-vVr7eJ_WywXDoV0p0l84ECwcQ4Qq1ozX1TBZb1ZzRZw6BwYP8n1TK9mx4k6YGOMSRYYG7Z2UyBDFs2tNFi1Cj3TMHPTADwF3wQjwRBnnyMjv1CG1_PN9vzUu9jjjbKSb3X6VVNqBQKrsROJxmh4CKxjA7C53QK1BO995xlxrUEx-giErP9ArxGZGOOeBg1uwAp8RilvJ6QDw07zuVUmz39bqS6eg
expires_at

In [16]:
er.request.__dict__

{'method': 'POST',
 'url': 'https://smartfoods.develop.tapis.io/v3/oauth2/tokens',
 'headers': {'Content-Type': 'application/json', 'Content-Length': '88'},
 '_cookies': <RequestsCookieJar[]>,
 'body': '{"grant_type": "device_code", "device_code": "R3OPUI6geB9rwy2l8hUM6oaFJYdTvucJS2phUEQs"}',
 'hooks': {'response': []},
 '_body_position': None}

In [56]:
t.authenticator.create_token.op_desc['requestBody']['content']['application/json']['schema']['properties'].keys()

dict_keys(['username', 'password', 'grant_type', 'redirect_uri', 'code', 'device_code', 'refresh_token'])

In [24]:
print(test_client.client_id)

Oa9aD3zr2vJ0g


In [25]:
client_code_info = t.authenticator.generate_device_code(client_id=test_client.client_id)
client_code_info


client_id: Oa9aD3zr2vJ0g
device_code: YNkuRuBgAXoltULD4hPgpC8nYiOezZYjjMzJqRtd
expires_in: Mon, 03 Jul 2023 18:38:42 GMT
user_code: UvggUJru
verification_uri: http://smartfoods.develop.tapis.io/v3/oauth2/device?client_id=Oa9aD3zr2vJ0g

In [28]:
token = t.authenticator.create_token(grant_type="device_code", device_code=client_code_info.device_code, client_id=test_client.client_id)

BadRequestError: message: Required client_id parameter missing.

In [10]:
t.authenticator.generate_device_code(client_id = test_client.client_id)

ServerDownError: message: Unrecognized exception type: <class 'AttributeError'>. Exception: '_AppCtxGlobals' object has no attribute 'token_claims'

In [4]:
url = r"https://smartfoods.tapis.io/v3/oauth2/device/code"

In [11]:
client_id = "OMgP9E9aaz3mL"

In [6]:
r = requests.post(url, json={'client_id':client_id})

In [7]:
print(r.content)

b'{"message":"Internal error saving device code. Please try again later.","metadata":{},"result":null,"status":"error","version":"dev"}\n'
